# Image classification

Code for this example comes from [Chapter 1](https://colab.research.google.com/github/fastai/fastbook/blob/master/01_intro.ipynb#scrollTo=51rjRXOLDugq) of
Deep Learning for Coders with fastai & PyTorch by Jeremy Howard and Sylvain Gugger.

## Set up

### Change the runtime

Change the Runtime to utilize the NVIDIA T4 GPU, if it is not already set to the GPU.

In the menu above click
Runtime $\rightarrow$ Change runtime type and change Hardware accelerator to T4 GPU

### Import the `fastai` library

Import all items from `fastai.vision.all`

In [ ]:
from fastai.vision.all import *

Import all items from `fastcore.all`

In [ ]:
from fastcore.all import *

## Data for dogs vs cats classifier

The data are originally from the [Oxford-IIIT Pet dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/), but we will be accessing the data from the fast.ai python library.

The dataset is available for commercial/research purposes under a [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/). The copyright remains with the original owners of the images.

Download and get the path to the images.

In [ ]:
path = untar_data(URLs.PETS)

##### Get the file names for each image

In [ ]:
file_names = get_image_files(path/'images')

In [ ]:
f'We have {len(file_names)} images'

##### Look at a few images

In [ ]:
# select an example image
img = PILImage.create(file_names[65]) # Change the number here to pick a different image

# plot the image
img.to_thumb(192)

## Data processing

### Process the image labels

Examine the image file names

In [ ]:
file_names[:5]

##### Define a function that labels cat and dog images according to a convention of those who put the data set together. Note that cat images have uppercase names and dogs have lowercase names:

So, we can define a function that produces the labels "cat" or "dog" according to this convention.

In [ ]:
def label_function(file_name):
    pass

### Process the images

Load the image data used to train and test the network.

First create a `DataBlock` that describes how the data should be loaded, transformed,  and what type of task it will be used for.

In [ ]:
dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock),
                   get_items = get_image_files,
                   get_y     = label_function,
                   splitter  = RandomSplitter(seed=1),
                   item_tfms = Resize(224))

Load the data

In [ ]:
dls = dblock.dataloaders(path/"images")

## Exploratory data analysis

Our exploratory data analysis consists of examining example images. We can use the `show_batch()` method to show a few images and their labels.


Think about how you can distinguish between images of dogs or cats. Can you describe a possible algorithm to classify the images?

In [ ]:
dls.show_batch(figsize=(6, 6))

## Modeling

We will use a deep neural network to build the classification model. Rather than training a network from scratch, we will modify an existing deep neural network to solve the problem of recognizing dogs vs. cats.

The model will be based on the ResNet-34 deep neural network that was trained on the [ImageNet](https://www.image-net.org/) dataset.

![picture](https://storage.googleapis.com/kaggle-datasets-images/6978/10038/d9e74a548a8cdca167b38520ac8bf405/data-original.png?t=2017-12-13-20-32-53)

The ImageNet data set consists of images of many types.

We will use transfer learning to train the network to classify images as dogs and cats.


In [ ]:
dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock),
                   get_items = get_image_files,
                   get_y     = label_function,
                   splitter  = RandomSplitter(seed=1),
                   item_tfms = Resize(224))

### Train and test the model

We will train a model to classify images as dogs or cats, starting from the resnet34 model that has already been trained to solve an image classification problem involing many types of images.

We create a `vision_learner` object by specifying:

(1) the data loader created above `dls`,

(2) the name of the neural network we want to use as the basis for the model `resnet34`, and

(3) the metric we want to use to evaluate the performance of the model `error_rate`.

We can then fine tune the weights of the model to perform our specific task by minimizing the error for a specified number of epochs.

### Assess model performance

fast.ai has a `ClassificationInterpretation` function to help analyze the performance of the model.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

Plot the confusion matrix, which shows the number of images in the validation set that were correctly and incorrectly classified by the model.

##### Show the top loss images

In [ ]:
interp.plot_top_losses(5, figsize=(8, 6))

## Deployment

We can use the model to predict the animal type for an example image.

In [ ]:
img = PILImage.create(file_names[500])

prediction,_,probs = learn.predict(img)

print(f"Predicted animal: {prediction}.")
print(f"Probability it is this animal is a dog =  {probs[1].item():.6f}")
img.to_thumb(192)

##### Try the the model with pictures of dogs or cats that you have of your own or find online.

Load a new dog or cat image to test the model. Try a picture of my dog:

In [ ]:
!wget -O Romi.png https://www.dropbox.com/scl/fi/nxxkxwzd19udfmkgq1vun/Romi_Workout.png?rlkey=m22qgnr1krt2p7u8ciick8mxl&dl=0

View the image

In [ ]:
img = PILImage.create('Romi.png')
img.to_thumb(300)

Predict the image class and plot the prediction

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
prediction, _, probs = learn.predict(img)

fig, axes = plt.subplots(1, 2, figsize=(8, 4))

# Display the image in the first subplot
axes[0].imshow(img)
axes[0].axis('off')
axes[0].set_title(f"Prediction: {prediction}")

# Display the bar plot in the second subplot
sns.barplot(x=probs, y=dls.vocab, orient='h', ax=axes[1])
axes[1].set_xlabel("Probability")
axes[1].set_ylabel("Class Label")
axes[1].set_title("Prediction Probabilities")

plt.tight_layout()
plt.show()

<pre>



















</pre>

##### Let's see what it predicts for a picture of [grumpy cat](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/Grumpy_Cat_%2814556024763%29_%28cropped%29.jpg/250px-Grumpy_Cat_%2814556024763%29_%28cropped%29.jpg) from Wikipedia

![picture](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/Grumpy_Cat_%2814556024763%29_%28cropped%29.jpg/250px-Grumpy_Cat_%2814556024763%29_%28cropped%29.jpg)



In [ ]:
# Download the image
!wget https://upload.wikimedia.org/wikipedia/commons/d/dc/Grumpy_Cat_%2814556024763%29_%28cropped%29.jpg -O grumpy.jpg

# create the image
img = PILImage.create('grumpy.jpg')

# plot the image
img.to_thumb(192)

In [ ]:
prediction, _, probs = learn.predict(img)

fig, axes = plt.subplots(1, 2, figsize=(8, 4))

# Display the image in the first subplot
axes[0].imshow(img)
axes[0].axis('off')
axes[0].set_title(f"Prediction: {prediction}")

# Display the bar plot in the second subplot
sns.barplot(x=probs, y=dls.vocab, orient='h', ax=axes[1])
axes[1].set_xlabel("Probability")
axes[1].set_ylabel("Class Label")
axes[1].set_title("Prediction Probabilities")

plt.tight_layout()
plt.show()

<pre>



















</pre>

## Evaluation

Do we fully understand how the model is classifying the images? Are you confident that the model will be robust?
